In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
np.random.seed(2018)
%matplotlib inline

/app/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
comments = pd.read_csv('data/reviews_Grocery_and_Gourmet_Food_5/aspect_Grocery_and_Gourmet_Food_5_reviewsentences.csv', names = ['Chat Aspects'])

In [4]:
len(comments)

89722

In [5]:
def CleanUp(sentence):
    #no_punc = [ch for ch in sentence if ch not in string.punctuation]
    #no_punc = [ch if ch not in string.punctuation else ' ' for ch in sentence]
    no_punc = [ch if (ch not in string.punctuation and ch.isdigit()==False) else ' ' for ch in sentence]
    no_punc = re.sub(' +',' ',''.join(no_punc).lower().strip())
    no_stop = [word for word in no_punc.split() if (word not in stopwords.words('english') and word.isdigit()==False)]
    ps = PorterStemmer()
    stem_word = [ps.stem(word) for word in no_stop]
    return(stem_word)
    #return(' '.join(stem_word))

In [11]:
comments_final = comments['Chat Aspects'].apply(CleanUp)
comments_final.head()

TypeError: 'float' object is not iterable

In [7]:
comments_final.head(20)

0               standard milk chocolate kit kat
1                   The green tea flavor subtle
2                       among Kit Kat fans well
3                   The green tea flavor subtle
4          green tea candies I tried. But still
5                      best Kit Kats I 've ever
6             I 've purchased different curries
7       Because Mae Ploy Thai Green Curry Paste
8              These dyes create awesome colors
9             couple drops really vibrant color
10                   natural / organic-I wish I
11      nice basic selection gel food colorings
12               The product seems high quality
13                      best energy bar I tried
14      better average tasting sports bar/snack
15         I 've listed nutritional information
16    PowerBar flavors well. Review M.  Reynard
17        PowerBar part regular morning routine
18          A couple times I 've asthma attacks
19                    high protein bars I tried
Name: Chat Aspects, dtype: object

In [109]:
len(comments_final)

551219

In [8]:
processed_docs = list(comments_final)

In [9]:
dictionary = gensim.corpora.Dictionary(comments_final)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [10]:
#dictionary.filter_extremes(no_below=100, no_above=0.5, keep_n=150000)
dictionary.filter_extremes(no_below=300, no_above=0.5, keep_n=150000)

NameError: name 'dictionary' is not defined

In [113]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus[4310]

In [114]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 31 ("call") appears 1 time.
Word 77 ("intern") appears 1 time.


In [115]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.8018761575907541), (1, 0.5974902743036811)]


In [120]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=4, iterations=300, minimum_probability=0.05, random_state=2018)

In [121]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.045*"b" + 0.042*"onlin" + 0.042*"set" + 0.033*"price" + 0.033*"app" + 0.028*"end" + 0.027*"alreadi" + 0.024*"x" + 0.021*"pin" + 0.021*"button"
Topic: 1 
Words: 0.179*"plan" + 0.079*"unlimit" + 0.068*"data" + 0.063*"time" + 0.062*"one" + 0.037*"gb" + 0.036*"go" + 0.034*"card" + 0.031*"chang" + 0.029*"question"
Topic: 2 
Words: 0.174*"today" + 0.133*"order" + 0.074*"place" + 0.034*"avail" + 0.032*"need" + 0.020*"id" + 0.019*"issu" + 0.019*"black" + 0.017*"year" + 0.017*"name"
Topic: 3 
Words: 0.173*"line" + 0.091*"fee" + 0.056*"charg" + 0.046*"upgrad" + 0.042*"activ" + 0.041*"right" + 0.038*"access" + 0.037*"option" + 0.033*"per" + 0.027*"sorri"
Topic: 4 
Words: 0.117*"bill" + 0.112*"new" + 0.112*"devic" + 0.079*"payment" + 0.041*"monthli" + 0.033*"current" + 0.031*"may" + 0.029*"credit" + 0.028*"call" + 0.028*"cycl"
Topic: 5 
Words: 0.083*"n" + 0.070*"month" + 0.056*"sure" + 0.055*"pleas" + 0.034*"email" + 0.032*"last" + 0.030*"provid" + 0.029*"address" + 0.027*"good"

In [76]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4, iterations=300, minimum_probability=0.05, random_state=2018)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.056*"digit" + 0.048*"secur" + 0.041*"code" + 0.037*"number" + 0.032*"verif" + 0.030*"pin" + 0.028*"mobil" + 0.026*"account" + 0.020*"discount" + 0.019*"free"
Topic: 1 Word: 0.053*"servic" + 0.049*"sorri" + 0.043*"date" + 0.038*"n" + 0.037*"know" + 0.034*"trade" + 0.031*"activ" + 0.029*"may" + 0.021*"today" + 0.017*"success"
Topic: 2 Word: 0.085*"plan" + 0.079*"unlimit" + 0.038*"go" + 0.034*"gb" + 0.031*"data" + 0.027*"verizon" + 0.026*"store" + 0.023*"account" + 0.020*"right" + 0.020*"intern"
Topic: 3 Word: 0.054*"line" + 0.042*"phone" + 0.035*"credit" + 0.031*"fee" + 0.031*"per" + 0.029*"return" + 0.023*"access" + 0.019*"issu" + 0.018*"bill" + 0.017*"via"
Topic: 4 Word: 0.064*"time" + 0.054*"payment" + 0.040*"ask" + 0.030*"may" + 0.028*"pass" + 0.026*"travel" + 0.021*"set" + 0.018*"week" + 0.015*"one" + 0.014*"devic"
Topic: 5 Word: 0.082*"bill" + 0.055*"chat" + 0.054*"verizon" + 0.035*"wireless" + 0.031*"sale" + 0.023*"button" + 0.022*"address" + 0.021*"cycl" + 0.018*

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index)))

In [122]:
unseen_document = 'Why should I pay extra $110 even though I haven\'t used any data'
bow_corpus1 = dictionary.doc2bow(CleanUp(unseen_document))
#tfidf = models.TfidfModel(bow_corpus)
#corpus_tfidf = tfidf[bow_corpus]
for index, score in sorted(lda_model[bow_corpus1], key=lambda tup: -1*tup[1]):
    #print(score, index)
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index)))

Score: 0.41746844602061717	 Topic: 0.065*"next" + 0.064*"discount" + 0.063*"bill" + 0.057*"due" + 0.046*"total" + 0.040*"pay" + 0.036*"’" + 0.034*"usag" + 0.034*"mobil" + 0.030*"protect"
Score: 0.18252313698927794	 Topic: 0.173*"line" + 0.091*"fee" + 0.056*"charg" + 0.046*"upgrad" + 0.042*"activ" + 0.041*"right" + 0.038*"access" + 0.037*"option" + 0.033*"per" + 0.027*"sorri"
Score: 0.15714714844977146	 Topic: 0.229*"phone" + 0.091*"number" + 0.051*"day" + 0.040*"like" + 0.034*"trade" + 0.032*"well" + 0.029*"use" + 0.021*"look" + 0.021*"back" + 0.020*"old"
Score: 0.15714296428120247	 Topic: 0.045*"b" + 0.042*"onlin" + 0.042*"set" + 0.033*"price" + 0.033*"app" + 0.028*"end" + 0.027*"alreadi" + 0.024*"x" + 0.021*"pin" + 0.021*"button"
